# Подготовка обучающей и тестовой выборки, кросс-валидация и подбор гиперпараметров на примере метода ближайших соседей

## Описание датасета

Датасет содержит информацию о результатах гонщиков Формулы 1.

Каждая запись в таблице содержит информацио о:
- имени пилота
- национальности пилота
- количестве чемпионских титулов
- количестве гонок
- количестве поул-позиций
- количестве побед в гонках
- количестве подиумов
- количестве быстрых кругов
- суммарном количестве очков
- проценте побед
- количестве лет выступления
- наличии статуса чемпиона

## Обработка датасета

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

In [4]:
data = pd.read_excel('./data/F1Drivers_Dataset.xlsx')

print(data)

                Driver     Nationality  Championships  Race_Entries  \
0          Carlo Abate           Italy            0.0           3.0   
1     George Abecassis  United Kingdom            0.0           2.0   
2        Kenny Acheson  United Kingdom            0.0          10.0   
3    Andrea de Adamich           Italy            0.0          36.0   
4       Philippe Adams         Belgium            0.0           2.0   
..                 ...             ...            ...           ...   
863      Emilio Zapico           Spain            0.0           1.0   
864        Zhou Guanyu           China            0.0          23.0   
865      Ricardo Zonta          Brazil            0.0          37.0   
866        Renzo Zorzi           Italy            0.0           7.0   
867     Ricardo Zunino       Argentina            0.0          11.0   

     Pole_Positions  Race_Wins  Podiums  Fastest_Laps  Points  Active  Decade  \
0               0.0        0.0      0.0           0.0     0.0   Fa

### Удаление пропусков в данных

### Кодирование категориальных признаков

## Работа с датасетом

### Разделение выборки на обучающую и тестовую

### Обучение модели